In [8]:
import requests

api_key='gaKMkjL1JzRhLGQ8pofFgbAN8wRMQNHcT0G96xr5'

# Step 1: Search for foods by keyword (e.g., "chicken")
search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
params = {
    'query': 'chicken raw thigh no skin',  # Search term
    'api_key': api_key
}

# Perform the search
response = requests.get(search_url, params=params)
data = response.json()

# Display a list of foods with their descriptions and FDC IDs
print("Search Results:")
for i, food in enumerate(data['foods']):
    print(f"{i+1}: {food['description']} (FDC ID: {food['fdcId']})")

# Step 2: Assume the user selects a row (e.g., row 1) from the list
# Select the first food item (or any other based on user input)
selected_food = data['foods'][0]  # In this case, selecting the first item
fdc_id = selected_food['fdcId']  # Get the FDC ID of the selected food

print(f"\nFetching nutrition details for: {selected_food['description']} (FDC ID: {fdc_id})")

# Step 3: Fetch detailed nutrition info using the FDC ID
details_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
details_response = requests.get(details_url, params={'api_key': api_key})
food_details = details_response.json()

# Step 4: Display detailed nutrition info

print(f"Name: {food_details['description']}")



Search Results:
1: Chicken, skin (drumsticks and thighs), raw (FDC ID: 172855)
2: Chicken, broilers or fryers, thigh, meat and skin, raw (FDC ID: 172385)
3: Chicken, skin (drumsticks and thighs), with added solution, raw (FDC ID: 174502)
4: Chicken thigh, stewed, skin eaten (FDC ID: 2341443)
5: Chicken thigh, sauteed, skin not eaten (FDC ID: 2341450)
6: Chicken thigh, rotisserie, skin eaten (FDC ID: 2341441)
7: Chicken thigh, rotisserie, skin not eaten (FDC ID: 2341442)
8: Chicken thigh, sauteed, skin eaten (FDC ID: 2341449)
9: Chicken thigh, stewed, skin not eaten (FDC ID: 2341444)
10: Chicken thigh, baked, broiled, or roasted, skin eaten, from raw (FDC ID: 2341435)
11: Chicken thigh, fried, coated, skin / coating eaten, from raw (FDC ID: 2341451)
12: Chicken thigh, fried, coated, skin / coating not eaten, from raw (FDC ID: 2341452)
13: Chicken thigh, baked, broiled, or roasted, skin not eaten, from raw (FDC ID: 2341436)
14: Chicken, dark meat, thigh, meat and skin, with added solutio

In [18]:
data

{'totalHits': 29930,
 'currentPage': 1,
 'totalPages': 599,
 'pageList': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'foodSearchCriteria': {'query': 'chicken raw thigh no skin',
  'generalSearchInput': 'chicken raw thigh no skin',
  'pageNumber': 1,
  'numberOfResultsPerPage': 50,
  'pageSize': 50,
  'requireAllWords': False},
 'foods': [{'fdcId': 172855,
   'description': 'Chicken, skin (drumsticks and thighs), raw',
   'commonNames': '',
   'additionalDescriptions': '',
   'dataType': 'SR Legacy',
   'ndbNumber': 5674,
   'publishedDate': '2019-04-01',
   'foodCategory': 'Poultry Products',
   'allHighlightFields': '',
   'score': 1186.4567,
   'microbes': [],
   'foodNutrients': [{'nutrientId': 1004,
     'nutrientName': 'Total lipid (fat)',
     'nutrientNumber': '204',
     'unitName': 'G',
     'derivationCode': 'A',
     'derivationDescription': 'Analytical',
     'derivationId': 1,
     'value': 44.2,
     'foodNutrientSourceId': 1,
     'foodNutrientSourceCode': '1',
     'foodNutrientS

In [27]:
food_details

{'fdcId': 172855,
 'description': 'Chicken, skin (drumsticks and thighs), raw',
 'publicationDate': '4/1/2019',
 'foodNutrients': [{'nutrient': {'id': 2045,
    'number': '951',
    'name': 'Proximates',
    'rank': 50,
    'isNutrientLabel': True,
    'indentLevel': 0,
    'numberOfDecimals': 3,
    'shortestName': 'Proximates',
    'unitName': 'g'},
   'type': 'FoodNutrient'},
  {'type': 'FoodNutrient',
   'nutrient': {'id': 1051,
    'number': '255',
    'name': 'Water',
    'rank': 100,
    'unitName': 'g'},
   'foodNutrientDerivation': {'id': 1,
    'code': 'A',
    'description': 'Analytical',
    'foodNutrientSource': {'id': 1,
     'code': '1',
     'description': 'Analytical or derived from analytical'}},
   'id': 1729960,
   'amount': 44.91,
   'dataPoints': 5,
   'max': 47.62,
   'min': 40.44},
  {'type': 'FoodNutrient',
   'nutrient': {'id': 1008,
    'number': '208',
    'name': 'Energy',
    'rank': 300,
    'unitName': 'kcal'},
   'foodNutrientDerivation': {'id': 49,
   

In [39]:
unit_dict = {
    "g": 1,
    "mg": 0.001,
    "µg": 0.000001,
    "kcal": 0.00000001,
    "kJ": 0.00000001,
    "IU": 0.0000001
}

tot_amt=0
for nutrient in food_details['foodNutrients']:
    #print(nutrient)
    #print(nutrient['nutrient']['name'])
    
    if 'amount' in nutrient:
        nutrient_amt=nutrient['amount']
        nutrient_unit=nutrient['nutrient']['unitName']
        nutrient_name=nutrient['nutrient']['name']  
        print(f"{nutrient_name}: {nutrient_amt} {nutrient_unit}")
        tot_amt += float(nutrient['amount']) * unit_dict[nutrient_unit]
        print(f"sub-total: {tot_amt}")

print(f"Total: {tot_amt}")


Water: 44.91 g
sub-total: 44.91
Energy: 440.0 kcal
sub-total: 44.9100044
Energy: 1839.0 kJ
sub-total: 44.91002279
Protein: 9.58 g
sub-total: 54.49002279
Total lipid (fat): 44.23 g
sub-total: 98.72002279
Ash: 0.49 g
sub-total: 99.21002279
Carbohydrate, by difference: 0.79 g
sub-total: 100.00002279
Fiber, total dietary: 0.0 g
sub-total: 100.00002279
Total Sugars: 0.0 g
sub-total: 100.00002279
Calcium, Ca: 6.0 mg
sub-total: 100.00602279
Iron, Fe: 0.37 mg
sub-total: 100.00639279
Magnesium, Mg: 8.0 mg
sub-total: 100.01439279
Phosphorus, P: 95.0 mg
sub-total: 100.10939279
Potassium, K: 119.0 mg
sub-total: 100.22839279
Sodium, Na: 51.0 mg
sub-total: 100.27939279
Zinc, Zn: 0.65 mg
sub-total: 100.28004279
Copper, Cu: 0.038 mg
sub-total: 100.28008079
Manganese, Mn: 0.014 mg
sub-total: 100.28009478999999
Selenium, Se: 10.1 µg
sub-total: 100.28010488999999
Vitamin C, total ascorbic acid: 0.0 mg
sub-total: 100.28010488999999
Thiamin: 0.04 mg
sub-total: 100.28014488999999
Riboflavin: 0.032 mg
sub-to